In [1]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

In [2]:
train_clientes = pd.read_csv('_data/train_clientes.csv')
train_requerimientos = pd.read_csv('_data/train_requerimientos.csv')
test_clientes = pd.read_csv('_data/test_clientes.csv')
test_requerimientos = pd.read_csv('_data/test_requerimientos.csv')

In [3]:
train = pd.read_csv('_data/train_merge.csv')
test = pd.read_csv('_data/test_merge.csv')

In [4]:
results = pd.read_csv('_data/results.csv')
results

model  accuracy  accuracy-std  f1-score  f1-score-std  \
0       XGBClassifier  0.872071      0.000393  0.413881      0.004304   
1      LGBMClassifier  0.876386      0.001123  0.481487      0.007929   
2  CatBoostClassifier  0.879357      0.000520  0.482438      0.006846   
3      TESTING_PARAMS  0.000000      0.000000  0.000000      0.000000   
4       XGBClassifier  0.880171      0.000859  0.484674      0.003495   
5       XGBClassifier  0.874900      0.001844  0.535230      0.003263   
6        FULL_DATASET  0.000000      0.000000  0.000000      0.000000   
7       XGBClassifier  0.872129      0.001760  0.411888      0.012084   
8      LGBMClassifier  0.877229      0.002107  0.474719      0.009287   

     aucroc  aucroc-std   logloss  logloss-std    time  
0  0.635150    0.002001  4.418511     0.013572   24861  
1  0.670094    0.005204  4.269507     0.038777    1674  
2  0.668718    0.003847  4.166874     0.017965  144761  
3  0.000000    0.000000  0.000000     0.000000       0  
4  0.669501    0.002300  4.138749     0.029675   31504  
5  0.707799    0.001611  4.320834     0.063708  113529  
6  0.000000    0.000000  0.000000     0.000000       0  
7  0.634202    0.005446  4.416538     0.060778  186127  
8  0.665458    0.004575  4.240392     0.072760    9602

In [7]:
train.shape

(70000, 85)

In [139]:
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows',150)

In [10]:
train.head()

ID_CORRELATIVO  FLG_BANCARIZADO  RANG_INGRESO  FLAG_LIMA_PROVINCIA  EDAD  \
0           35653                1             5                    0  25.0   
1           66575                1             2                    1  27.0   
2           56800                1             0                    1  34.0   
3            8410                1             3                    1  63.0   
4            6853                1            -1                    0  25.0   

   ANTIGUEDAD  ATTRITION  RANG_SDO_PASIVO_MENOS0  SDO_ACTIVO_MENOS0  \
0         6.0          0                       9                  0   
1         0.0          0                       1                  0   
2         4.0          0                       2                  0   
3         5.0          0                       3                  0   
4         0.0          0                       1                  0   

   SDO_ACTIVO_MENOS1  SDO_ACTIVO_MENOS2  SDO_ACTIVO_MENOS3  SDO_ACTIVO_MENOS4  \
0                  0                  0                  0                  0   
1                  0                  0                  0                  0   
2                  0                  0                  0                  0   
3                  0                  0                  0                  0   
4                  0                  0                  0                  0   

   SDO_ACTIVO_MENOS5  FLG_SEGURO_MENOS0  FLG_SEGURO_MENOS1  FLG_SEGURO_MENOS2  \
0                  0                  1                  1                  1   
1                  0                  0                  0                  0   
2                  0                  0                  0                  0   
3                  0                  1                  1                  1   
4                  0                  0                  0                  0   

   FLG_SEGURO_MENOS3  FLG_SEGURO_MENOS4  FLG_SEGURO_MENOS5  \
0                  1                  1                  1   
1                  0                  0                  0   
2                  0                  0                  0   
3                  1                  1                  1   
4                  0                  0                  0   

   RANG_NRO_PRODUCTOS_MENOS0  FLG_NOMINA  NRO_ACCES_CANAL1_MENOS0  \
0                          3           0                        2   
1                          1           1                        0   
2                          1           1                        0   
3                          2           0                        0   
4                          0           0                        0   

   NRO_ACCES_CANAL1_MENOS1  NRO_ACCES_CANAL1_MENOS2  NRO_ACCES_CANAL1_MENOS3  \
0                        1                        0                        2   
1                        0                        0                        0   
2                        0                        0                        0   
3                        0                        0                        0   
4                        0                        0                        0   

   NRO_ACCES_CANAL1_MENOS4  NRO_ACCES_CANAL1_MENOS5  NRO_ACCES_CANAL2_MENOS0  \
0                        4                        1                        8   
1                        0                        0                        0   
2                        0                        0                        0   
3                        0                        0                        0   
4                        0                        0                        0   

   NRO_ACCES_CANAL2_MENOS1  NRO_ACCES_CANAL2_MENOS2  NRO_ACCES_CANAL2_MENOS3  \
0                       11                       14                       12   
1                        0                        0                        0   
2                        0                        0                        0   
3                        0                        0      

In [30]:
null_cols = [column for column in train.columns if train[column].isnull().sum()>0]

In [31]:
for col in null_cols:
    train[f'{col}-isnull'] = train[col].isnull().astype(int)
    test[f'{col}-isnull'] = test[col].isnull().astype(int)

In [32]:
train.head()

ID_CORRELATIVO  FLG_BANCARIZADO  RANG_INGRESO  FLAG_LIMA_PROVINCIA  EDAD  \
0           35653                1             5                    0  25.0   
1           66575                1             2                    1  27.0   
2           56800                1             0                    1  34.0   
3            8410                1             3                    1  63.0   
4            6853                1            -1                    0  25.0   

   ANTIGUEDAD  ATTRITION  RANG_SDO_PASIVO_MENOS0  SDO_ACTIVO_MENOS0  \
0         6.0          0                       9                  0   
1         0.0          0                       1                  0   
2         4.0          0                       2                  0   
3         5.0          0                       3                  0   
4         0.0          0                       1                  0   

   SDO_ACTIVO_MENOS1  SDO_ACTIVO_MENOS2  SDO_ACTIVO_MENOS3  SDO_ACTIVO_MENOS4  \
0                  0                  0                  0                  0   
1                  0                  0                  0                  0   
2                  0                  0                  0                  0   
3                  0                  0                  0                  0   
4                  0                  0                  0                  0   

   SDO_ACTIVO_MENOS5  FLG_SEGURO_MENOS0  FLG_SEGURO_MENOS1  FLG_SEGURO_MENOS2  \
0                  0                  1                  1                  1   
1                  0                  0                  0                  0   
2                  0                  0                  0                  0   
3                  0                  1                  1                  1   
4                  0                  0                  0                  0   

   FLG_SEGURO_MENOS3  FLG_SEGURO_MENOS4  FLG_SEGURO_MENOS5  \
0                  1                  1                  1   
1                  0                  0                  0   
2                  0                  0                  0   
3                  1                  1                  1   
4                  0                  0                  0   

   RANG_NRO_PRODUCTOS_MENOS0  FLG_NOMINA  NRO_ACCES_CANAL1_MENOS0  \
0                          3           0                        2   
1                          1           1                        0   
2                          1           1                        0   
3                          2           0                        0   
4                          0           0                        0   

   NRO_ACCES_CANAL1_MENOS1  NRO_ACCES_CANAL1_MENOS2  NRO_ACCES_CANAL1_MENOS3  \
0                        1                        0                        2   
1                        0                        0                        0   
2                        0                        0                        0   
3                        0                        0                        0   
4                        0                        0                        0   

   NRO_ACCES_CANAL1_MENOS4  NRO_ACCES_CANAL1_MENOS5  NRO_ACCES_CANAL2_MENOS0  \
0                        4                        1                        8   
1                        0                        0                        0   
2                        0                        0                        0   
3                        0                        0                        0   
4                        0                        0                        0   

   NRO_ACCES_CANAL2_MENOS1  NRO_ACCES_CANAL2_MENOS2  NRO_ACCES_CANAL2_MENOS3  \
0                       11                       14                       12   
1                        0                        0                        0   
2                        0                        0                        0   
3                        0                        0      

In [33]:
# average mean and median
fill_EDAD = (train['EDAD'].dropna().mean() + test['EDAD'].dropna().median())/2
train['EDAD'] = train['EDAD'].fillna(fill_EDAD)
test['EDAD'] = test['EDAD'].fillna(fill_EDAD)

In [34]:
# average mean and median
fill_ANTIGUEDAD = (train['ANTIGUEDAD'].dropna().mean() + test['ANTIGUEDAD'].dropna().median())/2
train['ANTIGUEDAD'] = train['ANTIGUEDAD'].fillna(fill_ANTIGUEDAD)
test['ANTIGUEDAD'] = test['ANTIGUEDAD'].fillna(fill_ANTIGUEDAD)

In [40]:
train = train.fillna(0)
test = test.fillna(0)

In [44]:
train.to_csv('_data/train_full.csv', index=None)
test.to_csv('_data/test_full.csv', index=None)

In [9]:
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
import time
import winsound

from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, log_loss
from sklearn.model_selection import StratifiedKFold

In [10]:
kfolds = StratifiedKFold(n_splits=5, shuffle=True, random_state=1)

In [11]:
def run_model(model, X, y):
    model_name = str(model.__class__).split('.')[-1].replace('>','').replace("'",'')
    print(model_name)
    accuracy = []
    f1 = []
    auc = []
    logloss = []
    start = int(time.time() * 1000)
    for train_idx, test_idx in kfolds.split(X, y):
        model.fit(X.iloc[train_idx,:], y[train_idx])
        y_ = model.predict_proba(X.iloc[test_idx,:])
        accuracy += [accuracy_score(y.iloc[test_idx], y_)]
        f1 += [f1_score(y.iloc[test_idx], y_)]
        auc += [roc_auc_score(y.iloc[test_idx], y_)]
        logloss += [log_loss(y.iloc[test_idx], y_)]
    end = int(time.time() * 1000)
    results.loc[len(results)] = [model_name, np.mean(accuracy), np.std(accuracy), 
                                 np.mean(f1), np.std(f1),
                                 np.mean(auc), np.std(auc), 
                                 np.mean(logloss), np.std(logloss), str(end - start)]
    winsound.Beep(2500, 1000)

In [52]:
results.loc[len(results)] = ['NONULLS_DATASET'] + [0] * 9
results

model  accuracy  accuracy-std  f1-score  f1-score-std  \
0       XGBClassifier  0.872071      0.000393  0.413881      0.004304   
1      LGBMClassifier  0.876386      0.001123  0.481487      0.007929   
2  CatBoostClassifier  0.879357      0.000520  0.482438      0.006846   
3      TESTING_PARAMS  0.000000      0.000000  0.000000      0.000000   
4       XGBClassifier  0.880171      0.000859  0.484674      0.003495   
5       XGBClassifier  0.874900      0.001844  0.535230      0.003263   
6        FULL_DATASET  0.000000      0.000000  0.000000      0.000000   
7       XGBClassifier  0.872129      0.001760  0.411888      0.012084   
8      LGBMClassifier  0.877229      0.002107  0.474719      0.009287   
9     NONULLS_DATASET  0.000000      0.000000  0.000000      0.000000   

     aucroc  aucroc-std   logloss  logloss-std    time  
0  0.635150    0.002001  4.418511     0.013572   24861  
1  0.670094    0.005204  4.269507     0.038777    1674  
2  0.668718    0.003847  4.166874     0.017965  144761  
3  0.000000    0.000000  0.000000     0.000000       0  
4  0.669501    0.002300  4.138749     0.029675   31504  
5  0.707799    0.001611  4.320834     0.063708  113529  
6  0.000000    0.000000  0.000000     0.000000       0  
7  0.634202    0.005446  4.416538     0.060778  186127  
8  0.665458    0.004575  4.240392     0.072760    9602  
9  0.000000    0.000000  0.000000     0.000000       0

In [53]:
run_model(XGBClassifier(), train.drop(['ID_CORRELATIVO', 'ATTRITION'], axis=1), train['ATTRITION'])
results

XGBClassifier


model  accuracy  accuracy-std  f1-score  f1-score-std  \
0        XGBClassifier  0.872071      0.000393  0.413881      0.004304   
1       LGBMClassifier  0.876386      0.001123  0.481487      0.007929   
2   CatBoostClassifier  0.879357      0.000520  0.482438      0.006846   
3       TESTING_PARAMS  0.000000      0.000000  0.000000      0.000000   
4        XGBClassifier  0.880171      0.000859  0.484674      0.003495   
5        XGBClassifier  0.874900      0.001844  0.535230      0.003263   
6         FULL_DATASET  0.000000      0.000000  0.000000      0.000000   
7        XGBClassifier  0.872129      0.001760  0.411888      0.012084   
8       LGBMClassifier  0.877229      0.002107  0.474719      0.009287   
9      NONULLS_DATASET  0.000000      0.000000  0.000000      0.000000   
10       XGBClassifier  0.871157      0.000890  0.404228      0.009942   

      aucroc  aucroc-std   logloss  logloss-std    time  
0   0.635150    0.002001  4.418511     0.013572   24861  
1   0.670094    0.005204  4.269507     0.038777    1674  
2   0.668718    0.003847  4.166874     0.017965  144761  
3   0.000000    0.000000  0.000000     0.000000       0  
4   0.669501    0.002300  4.138749     0.029675   31504  
5   0.707799    0.001611  4.320834     0.063708  113529  
6   0.000000    0.000000  0.000000     0.000000       0  
7   0.634202    0.005446  4.416538     0.060778  186127  
8   0.665458    0.004575  4.240392     0.072760    9602  
9   0.000000    0.000000  0.000000     0.000000       0  
10  0.630796    0.004533  4.450089     0.030751   99903

In [54]:
run_model(LGBMClassifier(), train.drop(['ID_CORRELATIVO', 'ATTRITION'], axis=1), train['ATTRITION'])
results

LGBMClassifier


model  accuracy  accuracy-std  f1-score  f1-score-std  \
0        XGBClassifier  0.872071      0.000393  0.413881      0.004304   
1       LGBMClassifier  0.876386      0.001123  0.481487      0.007929   
2   CatBoostClassifier  0.879357      0.000520  0.482438      0.006846   
3       TESTING_PARAMS  0.000000      0.000000  0.000000      0.000000   
4        XGBClassifier  0.880171      0.000859  0.484674      0.003495   
5        XGBClassifier  0.874900      0.001844  0.535230      0.003263   
6         FULL_DATASET  0.000000      0.000000  0.000000      0.000000   
7        XGBClassifier  0.872129      0.001760  0.411888      0.012084   
8       LGBMClassifier  0.877229      0.002107  0.474719      0.009287   
9      NONULLS_DATASET  0.000000      0.000000  0.000000      0.000000   
10       XGBClassifier  0.871157      0.000890  0.404228      0.009942   
11      LGBMClassifier  0.876772      0.001658  0.472491      0.006863   

      aucroc  aucroc-std   logloss  logloss-std    time  
0   0.635150    0.002001  4.418511     0.013572   24861  
1   0.670094    0.005204  4.269507     0.038777    1674  
2   0.668718    0.003847  4.166874     0.017965  144761  
3   0.000000    0.000000  0.000000     0.000000       0  
4   0.669501    0.002300  4.138749     0.029675   31504  
5   0.707799    0.001611  4.320834     0.063708  113529  
6   0.000000    0.000000  0.000000     0.000000       0  
7   0.634202    0.005446  4.416538     0.060778  186127  
8   0.665458    0.004575  4.240392     0.072760    9602  
9   0.000000    0.000000  0.000000     0.000000       0  
10  0.630796    0.004533  4.450089     0.030751   99903  
11  0.664470    0.004438  4.256180     0.057268    5080

In [8]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

In [58]:
X_train, X_test, y_train, y_test = train_test_split(train.drop('ATTRITION', axis=1), train['ATTRITION'], 
                                                    test_size=0.2, stratify=train['ATTRITION'])

In [59]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((56000, 119), (14000, 119), (56000,), (14000,))

In [111]:
plt.subplots(ncols=2, figsize=(20,5))
plt.subplot(121)
sns.countplot(x=y_train)
plt.subplot(122)
sns.countplot(x=y_test)

In [63]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier

In [116]:
X_train_churn = X_train[y_train==1]
y_train_churn = y_train[y_train==1]
X_train_no_churn = X_train[y_train==0]
y_train_no_churn = y_train[y_train==0]

In [123]:
X_train_churn.shape, X_train_no_churn.shape, y_train_churn.shape, y_train_no_churn.shape

((8658, 119), (47342, 119), (8658,), (47342,))

In [134]:
xgb = XGBClassifier()
xgb.fit(X_train, y_train)
# xgb.fit(np.vstack((X_train_churn, X_train_no_churn[:10000])), 
#         np.concatenate((y_train_churn, y_train_no_churn[:10000])))

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=1, nthread=None, objective='binary:logistic', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1)

In [135]:
y_train_pred = xgb.predict(X_train)
print(accuracy_score(y_train, y_train_pred))
print(f1_score(y_train, y_train_pred))
print(roc_auc_score(y_train, y_train_pred))
print(log_loss(y_train, y_train_pred))
confusion_matrix(y_train, y_train_pred)

0.872625
0.413452841049
0.634738374599
4.39939076476


array([[46353,   989],
       [ 6144,  2514]], dtype=int64)

In [136]:
y_test_pred = xgb.predict(X_test)
print(accuracy_score(y_test, y_test_pred))
print(f1_score(y_test, y_test_pred))
print(roc_auc_score(y_test, y_test_pred))
print(log_loss(y_test, y_test_pred))
confusion_matrix(y_test, y_test_pred)

0.871428571429
0.40906106369
0.633005199214
4.44071461476


array([[11577,   259],
       [ 1541,   623]], dtype=int64)

##### > Conclusion: underfitting

## Outlier clipping

In [147]:
train = pd.read_csv('_data/train_full.csv')
test = pd.read_csv('_data/test_full.csv')

In [148]:
train.describe(percentiles=[.01, 0.25, 0.50, 0.75, 0.99]).transpose()

count          mean           std  \
ID_CORRELATIVO                          70000.0  49922.740729  28896.279736   
FLG_BANCARIZADO                         70000.0      0.839043      0.367494   
RANG_INGRESO                            70000.0      1.767600      2.207242   
FLAG_LIMA_PROVINCIA                     70000.0      0.308443      0.556824   
EDAD                                    70000.0     35.109501     11.096763   
ANTIGUEDAD                              70000.0      2.517199      3.171116   
ATTRITION                               70000.0      0.154600      0.361526   
RANG_SDO_PASIVO_MENOS0                  70000.0      2.762586      3.376407   
SDO_ACTIVO_MENOS0                       70000.0   9650.613286  99710.661676   
SDO_ACTIVO_MENOS1                       70000.0   9389.333714  98936.962773   
SDO_ACTIVO_MENOS2                       70000.0   9017.541143  96530.270110   
SDO_ACTIVO_MENOS3                       70000.0   8678.496714  94156.817724   
SDO_ACTIVO_MENOS4                       70000.0   8477.067714  87550.907245   
SDO_ACTIVO_MENOS5                       70000.0   8266.671714  89436.601260   
FLG_SEGURO_MENOS0                       70000.0      0.493914      0.499967   
FLG_SEGURO_MENOS1                       70000.0      0.458429      0.498272   
FLG_SEGURO_MENOS2                       70000.0      0.426000      0.494497   
FLG_SEGURO_MENOS3                       70000.0      0.365386      0.481542   
FLG_SEGURO_MENOS4                       70000.0      0.323971      0.467993   
FLG_SEGURO_MENOS5                       70000.0      0.310257      0.462602   
RANG_NRO_PRODUCTOS_MENOS0               70000.0      1.523014      0.893626   
FLG_NOMINA                              70000.0      0.385029      0.486606   
NRO_ACCES_CANAL1_MENOS0                 70000.0      1.035086      2.746253   
NRO_ACCES_CANAL1_MENOS1                 70000.0      1.020943      2.749709   
NRO_ACCES_CANAL1_MENOS2                 70000.0      0.867100      2.512787   
NRO_ACCES_CANAL1_MENOS3                 70000.0      0.891743      2.649147   
NRO_ACCES_CANAL1_MENOS4                 70000.0      0.813557      2.514842   
NRO_ACCES_CANAL1_MENOS5                 70000.0      0.826300      2.545633   
NRO_ACCES_CANAL2_MENOS0                 70000.0      1.786986      4.528780   
NRO_ACCES_CANAL2_MENOS1                 70000.0      1.739486      4.614673   
NRO_ACCES_CANAL2_MENOS2                 70000.0      1.318186      3.929470   
NRO_ACCES_CANAL2_MENOS3                 70000.0      1.255300      3.974264   
NRO_ACCES_CANAL2_MENOS4                 70000.0      1.120829      3.732211   
NRO_ACCES_CANAL2_MENOS5                 70000.0      1.123800      3.777387   
NRO_ACCES_CANAL3_MENOS0                 70000.0      5.558000      7.776502   
NRO_ACCES_CANAL3_MENOS1                 70000.0      5.830871      8.254698   
NRO_ACCES_CANAL3_MENOS2                 70000.0      4.551771      7.203200   
NRO_ACCES_CANAL3_MENOS3                 70000.0      4.146200      7.308573   
NRO_ACCES_CANAL3_MENOS4                 70000.0      3.565257      7.179745   
NRO_ACCES_CANAL3_MENOS5                 70000.0      3.550743      7.331213   
NRO_ENTID_SSFF_MENOS0                   70000.0      2.030643      1.932397   
NRO_ENTID_SSFF_MENOS1                   70000.0      1.999243      1.920776   
NRO_ENTID_SSFF_MENOS2                   70000.0      1.968014      1.907792   
NRO_ENTID_SSFF_MENOS3                   70000.0      1.928971      1.889000   
NRO_ENTID_SSFF_MENOS4                   70000.0      1.873600      1.869672   
NRO_ENTID_SSFF_MENOS5                   70000.0      1.854386      1.853515   
FLG_SDO_OTSSFF_MENOS0                   70000.0      0.538400      0.498527   
FLG_SDO_OTSSFF_MENOS1                   70000.0      0.536557      0.498665   
FLG_SDO_OTSSFF_MENOS2                   70000.0      0.537843      0.498569   
FLG_SDO_OTSSFF_MENOS3                   70000.0      0.535414      0.498748   
FLG_SDO_OTSSF

In [158]:
for column in test.columns[1:]:
    q01 = train[column].quantile(0.01)
    q99 = train[column].quantile(0.99)
    train[column] = train[column].apply(lambda x: np.clip(x, q01, q99))
    test[column] = test[column].apply(lambda x: np.clip(x, q01, q99))

In [160]:
train.describe(percentiles=[.01, 0.25, 0.50, 0.75, 0.99]).transpose()

count          mean           std  \
ID_CORRELATIVO                          70000.0  49922.740729  28896.279736   
FLG_BANCARIZADO                         70000.0      0.839043      0.367494   
RANG_INGRESO                            70000.0      1.767600      2.207242   
FLAG_LIMA_PROVINCIA                     70000.0      0.308443      0.556824   
EDAD                                    70000.0     35.057673     10.847235   
ANTIGUEDAD                              70000.0      2.515613      3.165204   
ATTRITION                               70000.0      0.154600      0.361526   
RANG_SDO_PASIVO_MENOS0                  70000.0      2.762586      3.376407   
SDO_ACTIVO_MENOS0                       70000.0   6084.876924  29431.925055   
SDO_ACTIVO_MENOS1                       70000.0   5820.464754  28555.657767   
SDO_ACTIVO_MENOS2                       70000.0   5531.623937  27812.321680   
SDO_ACTIVO_MENOS3                       70000.0   5266.690533  27073.345398   
SDO_ACTIVO_MENOS4                       70000.0   5128.963286  26634.194576   
SDO_ACTIVO_MENOS5                       70000.0   4933.682636  25791.292196   
FLG_SEGURO_MENOS0                       70000.0      0.493914      0.499967   
FLG_SEGURO_MENOS1                       70000.0      0.458429      0.498272   
FLG_SEGURO_MENOS2                       70000.0      0.426000      0.494497   
FLG_SEGURO_MENOS3                       70000.0      0.365386      0.481542   
FLG_SEGURO_MENOS4                       70000.0      0.323971      0.467993   
FLG_SEGURO_MENOS5                       70000.0      0.310257      0.462602   
RANG_NRO_PRODUCTOS_MENOS0               70000.0      1.522614      0.892292   
FLG_NOMINA                              70000.0      0.385029      0.486606   
NRO_ACCES_CANAL1_MENOS0                 70000.0      0.995129      2.493235   
NRO_ACCES_CANAL1_MENOS1                 70000.0      0.986600      2.535186   
NRO_ACCES_CANAL1_MENOS2                 70000.0      0.831129      2.278878   
NRO_ACCES_CANAL1_MENOS3                 70000.0      0.855943      2.416825   
NRO_ACCES_CANAL1_MENOS4                 70000.0      0.775229      2.264662   
NRO_ACCES_CANAL1_MENOS5                 70000.0      0.788229      2.303550   
NRO_ACCES_CANAL2_MENOS0                 70000.0      1.747486      4.331999   
NRO_ACCES_CANAL2_MENOS1                 70000.0      1.706571      4.448690   
NRO_ACCES_CANAL2_MENOS2                 70000.0      1.283243      3.734050   
NRO_ACCES_CANAL2_MENOS3                 70000.0      1.213957      3.741219   
NRO_ACCES_CANAL2_MENOS4                 70000.0      1.087700      3.534429   
NRO_ACCES_CANAL2_MENOS5                 70000.0      1.088114      3.564686   
NRO_ACCES_CANAL3_MENOS0                 70000.0      5.446843      7.093897   
NRO_ACCES_CANAL3_MENOS1                 70000.0      5.730000      7.678259   
NRO_ACCES_CANAL3_MENOS2                 70000.0      4.447943      6.603172   
NRO_ACCES_CANAL3_MENOS3                 70000.0      4.042443      6.705339   
NRO_ACCES_CANAL3_MENOS4                 70000.0      3.457829      6.470332   
NRO_ACCES_CANAL3_MENOS5                 70000.0      3.439257      6.633482   
NRO_ENTID_SSFF_MENOS0                   70000.0      2.023100      1.909904   
NRO_ENTID_SSFF_MENOS1                   70000.0      1.991714      1.898130   
NRO_ENTID_SSFF_MENOS2                   70000.0      1.960814      1.885907   
NRO_ENTID_SSFF_MENOS3                   70000.0      1.921957      1.867313   
NRO_ENTID_SSFF_MENOS4                   70000.0      1.867014      1.848878   
NRO_ENTID_SSFF_MENOS5                   70000.0      1.848043      1.833256   
FLG_SDO_OTSSFF_MENOS0                   70000.0      0.538400      0.498527   
FLG_SDO_OTSSFF_MENOS1                   70000.0      0.536557      0.498665   
FLG_SDO_OTSSFF_MENOS2                   70000.0      0.537843      0.498569   
FLG_SDO_OTSSFF_MENOS3                   70000.0      0.535414      0.498748   
FLG_SDO_OTSSF

In [164]:
train.shape, test.shape

((70000, 120), (30000, 119))

In [165]:
for col in test.columns[1:]:
    if train[col].sum() == 0:
        train = train.drop(col, axis=1)
        test = test.drop(col, axis=1)

In [166]:
train.shape, test.shape

((70000, 105), (30000, 104))

In [167]:
train.to_csv('_data/train_clipped.csv', index=False)
test.to_csv('_data/test_clipped.csv', index=False)

## Normalization

In [168]:
from sklearn.preprocessing import MinMaxScaler

In [169]:
sc = MinMaxScaler()

In [170]:
train_norm = sc.fit_transform(train.drop(['ID_CORRELATIVO', 'ATTRITION'], axis=1))

In [174]:
train[train.drop(['ID_CORRELATIVO', 'ATTRITION'], axis=1).columns] = train_norm

In [176]:
test_norm = sc.transform(test.drop('ID_CORRELATIVO', axis=1))

In [177]:
test[test.drop('ID_CORRELATIVO', axis=1).columns] = test_norm

In [180]:
train.to_csv('_data/train_norm.csv', index=False)
test.to_csv('_data/test_norm.csv', index=False)

## Checkpoint

In [5]:
train = pd.read_csv('_data/train_norm.csv')
test = pd.read_csv('_data/test_norm.csv')

In [6]:
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import RidgeClassifier
from sklearn.svm import SVC

In [12]:
X_train, X_test, y_train, y_test = train_test_split(train.drop('ATTRITION', axis=1), train['ATTRITION'], 
                                                    test_size=0.2, stratify=train['ATTRITION'])

In [19]:
xgb = CatBoostClassifier()
xgb.fit(X_train, y_train)
# xgb.fit(np.vstack((X_train_churn, X_train_no_churn[:10000])), 
#         np.concatenate((y_train_churn, y_train_no_churn[:10000])))

In [20]:
y_train_pred = xgb.predict_proba(X_train)[:,1]
# print(accuracy_score(y_train, y_train_pred))
# print(f1_score(y_train, y_train_pred))
print(roc_auc_score(y_train, y_train_pred))
print(log_loss(y_train, y_train_pred))
# confusion_matrix(y_train, y_train_pred)

0.87280047325
0.289666455241


In [21]:
y_test_pred = xgb.predict_proba(X_test)[:,1]
# print(accuracy_score(y_test, y_test_pred))
# print(f1_score(y_test, y_test_pred))
print(roc_auc_score(y_test, y_test_pred))
print(log_loss(y_test, y_test_pred))
# confusion_matrix(y_test, y_test_pred)

0.857286293766
0.302406657185


In [185]:
results.loc[len(results)] = ['NORMALIZED'] + [0] * 9
results

model  accuracy  accuracy-std  f1-score  f1-score-std  \
0        XGBClassifier  0.872071      0.000393  0.413881      0.004304   
1       LGBMClassifier  0.876386      0.001123  0.481487      0.007929   
2   CatBoostClassifier  0.879357      0.000520  0.482438      0.006846   
3       TESTING_PARAMS  0.000000      0.000000  0.000000      0.000000   
4        XGBClassifier  0.880171      0.000859  0.484674      0.003495   
5        XGBClassifier  0.874900      0.001844  0.535230      0.003263   
6         FULL_DATASET  0.000000      0.000000  0.000000      0.000000   
7        XGBClassifier  0.872129      0.001760  0.411888      0.012084   
8       LGBMClassifier  0.877229      0.002107  0.474719      0.009287   
9      NONULLS_DATASET  0.000000      0.000000  0.000000      0.000000   
10       XGBClassifier  0.871157      0.000890  0.404228      0.009942   
11      LGBMClassifier  0.876772      0.001658  0.472491      0.006863   
12          NORMALIZED  0.000000      0.000000  0.000000      0.000000   

      aucroc  aucroc-std   logloss  logloss-std    time  
0   0.635150    0.002001  4.418511     0.013572   24861  
1   0.670094    0.005204  4.269507     0.038777    1674  
2   0.668718    0.003847  4.166874     0.017965  144761  
3   0.000000    0.000000  0.000000     0.000000       0  
4   0.669501    0.002300  4.138749     0.029675   31504  
5   0.707799    0.001611  4.320834     0.063708  113529  
6   0.000000    0.000000  0.000000     0.000000       0  
7   0.634202    0.005446  4.416538     0.060778  186127  
8   0.665458    0.004575  4.240392     0.072760    9602  
9   0.000000    0.000000  0.000000     0.000000       0  
10  0.630796    0.004533  4.450089     0.030751   99903  
11  0.664470    0.004438  4.256180     0.057268    5080  
12  0.000000    0.000000  0.000000     0.000000       0

In [186]:
run_model(XGBClassifier(), train.drop(['ID_CORRELATIVO', 'ATTRITION'], axis=1), train['ATTRITION'])
results

XGBClassifier


model  accuracy  accuracy-std  f1-score  f1-score-std  \
0        XGBClassifier  0.872071      0.000393  0.413881      0.004304   
1       LGBMClassifier  0.876386      0.001123  0.481487      0.007929   
2   CatBoostClassifier  0.879357      0.000520  0.482438      0.006846   
3       TESTING_PARAMS  0.000000      0.000000  0.000000      0.000000   
4        XGBClassifier  0.880171      0.000859  0.484674      0.003495   
5        XGBClassifier  0.874900      0.001844  0.535230      0.003263   
6         FULL_DATASET  0.000000      0.000000  0.000000      0.000000   
7        XGBClassifier  0.872129      0.001760  0.411888      0.012084   
8       LGBMClassifier  0.877229      0.002107  0.474719      0.009287   
9      NONULLS_DATASET  0.000000      0.000000  0.000000      0.000000   
10       XGBClassifier  0.871157      0.000890  0.404228      0.009942   
11      LGBMClassifier  0.876772      0.001658  0.472491      0.006863   
12          NORMALIZED  0.000000      0.000000  0.000000      0.000000   
13       XGBClassifier  0.871757      0.000804  0.410670      0.012102   

      aucroc  aucroc-std   logloss  logloss-std    time  
0   0.635150    0.002001  4.418511     0.013572   24861  
1   0.670094    0.005204  4.269507     0.038777    1674  
2   0.668718    0.003847  4.166874     0.017965  144761  
3   0.000000    0.000000  0.000000     0.000000       0  
4   0.669501    0.002300  4.138749     0.029675   31504  
5   0.707799    0.001611  4.320834     0.063708  113529  
6   0.000000    0.000000  0.000000     0.000000       0  
7   0.634202    0.005446  4.416538     0.060778  186127  
8   0.665458    0.004575  4.240392     0.072760    9602  
9   0.000000    0.000000  0.000000     0.000000       0  
10  0.630796    0.004533  4.450089     0.030751   99903  
11  0.664470    0.004438  4.256180     0.057268    5080  
12  0.000000    0.000000  0.000000     0.000000       0  
13  0.633756    0.005602  4.429366     0.027781  114782

In [187]:
run_model(LGBMClassifier(), train.drop(['ID_CORRELATIVO', 'ATTRITION'], axis=1), train['ATTRITION'])
results

LGBMClassifier


model  accuracy  accuracy-std  f1-score  f1-score-std  \
0        XGBClassifier  0.872071      0.000393  0.413881      0.004304   
1       LGBMClassifier  0.876386      0.001123  0.481487      0.007929   
2   CatBoostClassifier  0.879357      0.000520  0.482438      0.006846   
3       TESTING_PARAMS  0.000000      0.000000  0.000000      0.000000   
4        XGBClassifier  0.880171      0.000859  0.484674      0.003495   
5        XGBClassifier  0.874900      0.001844  0.535230      0.003263   
6         FULL_DATASET  0.000000      0.000000  0.000000      0.000000   
7        XGBClassifier  0.872129      0.001760  0.411888      0.012084   
8       LGBMClassifier  0.877229      0.002107  0.474719      0.009287   
9      NONULLS_DATASET  0.000000      0.000000  0.000000      0.000000   
10       XGBClassifier  0.871157      0.000890  0.404228      0.009942   
11      LGBMClassifier  0.876772      0.001658  0.472491      0.006863   
12          NORMALIZED  0.000000      0.000000  0.000000      0.000000   
13       XGBClassifier  0.871757      0.000804  0.410670      0.012102   
14      LGBMClassifier  0.876772      0.001658  0.472491      0.006863   

      aucroc  aucroc-std   logloss  logloss-std    time  
0   0.635150    0.002001  4.418511     0.013572   24861  
1   0.670094    0.005204  4.269507     0.038777    1674  
2   0.668718    0.003847  4.166874     0.017965  144761  
3   0.000000    0.000000  0.000000     0.000000       0  
4   0.669501    0.002300  4.138749     0.029675   31504  
5   0.707799    0.001611  4.320834     0.063708  113529  
6   0.000000    0.000000  0.000000     0.000000       0  
7   0.634202    0.005446  4.416538     0.060778  186127  
8   0.665458    0.004575  4.240392     0.072760    9602  
9   0.000000    0.000000  0.000000     0.000000       0  
10  0.630796    0.004533  4.450089     0.030751   99903  
11  0.664470    0.004438  4.256180     0.057268    5080  
12  0.000000    0.000000  0.000000     0.000000       0  
13  0.633756    0.005602  4.429366     0.027781  114782  
14  0.664470    0.004438  4.256180     0.057268    5174

## Submission

In [23]:
train.head()

ID_CORRELATIVO  FLG_BANCARIZADO  RANG_INGRESO  FLAG_LIMA_PROVINCIA  \
0           35653              1.0      0.666667                  0.5   
1           66575              1.0      0.333333                  1.0   
2           56800              1.0      0.111111                  1.0   
3            8410              1.0      0.444444                  1.0   
4            6853              1.0      0.000000                  0.5   

       EDAD  ANTIGUEDAD  ATTRITION  RANG_SDO_PASIVO_MENOS0  SDO_ACTIVO_MENOS0  \
0  0.081633         0.6          0                0.642857                0.0   
1  0.122449         0.0          0                0.071429                0.0   
2  0.265306         0.4          0                0.142857                0.0   
3  0.857143         0.5          0                0.214286                0.0   
4  0.081633         0.0          0                0.071429                0.0   

   SDO_ACTIVO_MENOS1          ...           \
0                0.0          ...            
1                0.0          ...            
2                0.0          ...            
3                0.0          ...            
4                0.0          ...            

   PRODUCTO_SERVICIO_2-Producto 13-isnull  \
0                                     0.0   
1                                     1.0   
2                                     0.0   
3                                     0.0   
4                                     1.0   

   PRODUCTO_SERVICIO_2-Producto 23-isnull  \
0                                     0.0   
1                                     1.0   
2                                     0.0   
3                                     0.0   
4                                     1.0   

   PRODUCTO_SERVICIO_2-Producto 16-isnull  \
0                                     0.0   
1                                     1.0   
2                                     0.0   
3                                     0.0   
4                                     1.0   

   PRODUCTO_SERVICIO_2-Producto 15-isnull  CODMES-201206-isnull  \
0                                     0.0                   0.0   
1                                     1.0                   1.0   
2                                     0.0                   0.0   
3                                     0.0                   0.0   
4                                     1.0                   1.0   

   CODMES-201205-isnull  CODMES-201208-isnull  CODMES-201204-isnull  \
0                   0.0                   0.0                   0.0   
1                   1.0                   1.0                   1.0   
2                   0.0                   0.0                   0.0   
3                   0.0                   0.0                   0.0   
4                   1.0                   1.0                   1.0   

   CODMES-201203-isnull  CODMES-201207-isnull  
0                   0.0                   0.0  
1                   1.0                   1.0  
2                   0.0                   0.0  
3                   0.0                   0.0  
4                   1.0                   1.0  

[5 rows x 105 columns]

In [24]:
xgb = CatBoostClassifier()
xgb.fit(train.drop(['ID_CORRELATIVO', 'ATTRITION'], axis=1), train['ATTRITION'])

In [26]:
test.head()

ID_CORRELATIVO  FLG_BANCARIZADO  RANG_INGRESO  FLAG_LIMA_PROVINCIA  \
0           47411              0.0      0.333333                  0.0   
1           39861              1.0      0.333333                  1.0   
2           38898              1.0      0.444444                  0.5   
3           50927              1.0      0.222222                  0.5   
4           32969              1.0      0.333333                  0.5   

       EDAD  ANTIGUEDAD  RANG_SDO_PASIVO_MENOS0  SDO_ACTIVO_MENOS0  \
0  0.257474         0.0                0.071429                0.0   
1  0.326531         0.0                0.071429                0.0   
2  0.102041         0.4                0.285714                0.0   
3  0.265306         0.4                0.500000                0.0   
4  0.020408         0.2                0.071429                0.0   

   SDO_ACTIVO_MENOS1  SDO_ACTIVO_MENOS2          ...           \
0                0.0                0.0          ...            
1                0.0                0.0          ...            
2                0.0                0.0          ...            
3                0.0                0.0          ...            
4                0.0                0.0          ...            

   PRODUCTO_SERVICIO_2-Producto 13-isnull  \
0                                     1.0   
1                                     1.0   
2                                     0.0   
3                                     0.0   
4                                     0.0   

   PRODUCTO_SERVICIO_2-Producto 23-isnull  \
0                                     1.0   
1                                     1.0   
2                                     0.0   
3                                     0.0   
4                                     0.0   

   PRODUCTO_SERVICIO_2-Producto 16-isnull  \
0                                     1.0   
1                                     1.0   
2                                     0.0   
3                                     0.0   
4                                     0.0   

   PRODUCTO_SERVICIO_2-Producto 15-isnull  CODMES-201206-isnull  \
0                                     1.0                   1.0   
1                                     1.0                   1.0   
2                                     0.0                   0.0   
3                                     0.0                   0.0   
4                                     0.0                   0.0   

   CODMES-201205-isnull  CODMES-201208-isnull  CODMES-201204-isnull  \
0                   1.0                   1.0                   1.0   
1                   1.0                   1.0                   1.0   
2                   0.0                   0.0                   0.0   
3                   0.0                   0.0                   0.0   
4                   0.0                   0.0                   0.0   

   CODMES-201203-isnull  CODMES-201207-isnull  
0                   1.0                   1.0  
1                   1.0                   1.0  
2                   0.0                   0.0  
3                   0.0                   0.0  
4                   0.0                   0.0  

[5 rows x 104 columns]

In [28]:
y_test_pred = xgb.predict_proba(test.drop('ID_CORRELATIVO', axis=1))[:,1]
submission = pd.DataFrame()
submission['ID_CORRELATIVO'] = test['ID_CORRELATIVO']
submission['ATTRITION'] = y_test_pred

In [29]:
submission.head()

ID_CORRELATIVO  ATTRITION
0           47411   0.211313
1           39861   0.334787
2           38898   0.023138
3           50927   0.027316
4           32969   0.280967

In [30]:
submission.to_csv('submission.csv', index=False)